In [1]:
pip install pandas numpy wfdb requests tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 41.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 3.0.1 which is incompatible.
bqplot 0.12.45 requires pandas<3.0.0,>=1.0.0, but you have pandas 3.0.1 which is incompatible.
db-dtypes 1.5.0 requires pandas<3.0.0,>=1.5.3, but you have pandas 3.0.1 which is incompatible.
gradio 5.50.0 requires pandas<3.0,>=1.0, but you have pandas 3.0.1 which is incompatible.


In [3]:
import os
import numpy as np
import pandas as pd
import wfdb
import ast
import requests

In [4]:
!pip install wfdb

import wfdb
wfdb.dl_database('ptb-xl', dl_dir='/content/ptbxl')


Streaming output truncated to the last 5000 lines.
Generating list of all files for: records100/02000/02221_lr
Generating list of all files for: records100/02000/02222_lr
Generating list of all files for: records100/02000/02223_lr
Generating list of all files for: records100/02000/02224_lr
Generating list of all files for: records100/02000/02225_lr
Generating list of all files for: records100/02000/02226_lr
Generating list of all files for: records100/02000/02227_lr
Generating list of all files for: records100/02000/02228_lr
Generating list of all files for: records100/02000/02229_lr
Generating list of all files for: records100/02000/02230_lr
Generating list of all files for: records100/02000/02231_lr
Generating list of all files for: records100/02000/02232_lr
Generating list of all files for: records100/02000/02233_lr
Generating list of all files for: records100/02000/02234_lr
Generating list of all files for: records100/02000/02235_lr
Generating list of all files for: records100/0200

KeyboardInterrupt: 

In [8]:
import wfdb

wfdb.dl_database(
    'ptb-xl',
    dl_dir='/content/ptbxl',
    keep_subdirs=True
)

Streaming output truncated to the last 5000 lines.
Generating record list for: records500/18000/18263_hr
Generating record list for: records500/18000/18264_hr
Generating record list for: records500/18000/18265_hr
Generating record list for: records500/18000/18266_hr
Generating record list for: records500/18000/18267_hr
Generating record list for: records500/18000/18268_hr
Generating record list for: records500/18000/18269_hr
Generating record list for: records500/18000/18270_hr
Generating record list for: records500/18000/18271_hr
Generating record list for: records500/18000/18272_hr
Generating record list for: records500/18000/18273_hr
Generating record list for: records500/18000/18274_hr
Generating record list for: records500/18000/18275_hr
Generating record list for: records500/18000/18276_hr
Generating record list for: records500/18000/18277_hr
Generating record list for: records500/18000/18278_hr
Generating record list for: records500/18000/18279_hr
Generating record list for: rec

KeyboardInterrupt: 

In [25]:
# ==========================================
# IMPORTS
# ==========================================
import wfdb
import numpy as np
import pandas as pd
import ast
import requests
import os

In [26]:
# ==========================================
# CONFIG
# ==========================================
PTBXL_PATH = "/content/ptbxl/"
ECG_FILE = "records100/00000/00001_lr"

API_BASE = "https://mystic-cbk-ecg-fm-api.hf.space"
ANALYZE_URL = f"{API_BASE}/analyze"
FEATURE_URL = f"{API_BASE}/extract_features"
HEALTH_URL = f"{API_BASE}/health"


In [27]:
# ==========================================
# STEP 1: LOAD METADATA
# ==========================================
print("Loading metadata...")
meta_df = pd.read_csv(
    os.path.join(PTBXL_PATH, "ptbxl_database.csv"),
    index_col="ecg_id"
)
meta_df.scp_codes = meta_df.scp_codes.apply(lambda x: ast.literal_eval(x))
print("Metadata loaded.")

Loading metadata...
Metadata loaded.


In [28]:
# ==========================================
# STEP 2: LOAD ECG SIGNAL
# ==========================================
print("\nLoading ECG signal...")
signal, fields = wfdb.rdsamp(os.path.join(PTBXL_PATH, ECG_FILE))

print("Raw ECG shape:", signal.shape)  # (1000, 12)


Loading ECG signal...
Raw ECG shape: (1000, 12)


In [29]:
# ==========================================
# STEP 3: SHOW TRUE LABEL
# ==========================================
record_id = int(ECG_FILE.split("/")[-1].split("_")[0])
true_label = meta_df.loc[record_id].scp_codes
print("\nTrue SCP Codes:", true_label)



True SCP Codes: {'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}


In [30]:
# ==========================================
# STEP 4: PREPROCESS SIGNAL
# ==========================================
def preprocess_ecg(signal):
    # Normalize per lead
    signal = (signal - np.mean(signal, axis=0)) / np.std(signal, axis=0)

    # Transpose to (12, 1000) for Conv1D
    signal = signal.T

    return signal.tolist()

processed_signal = preprocess_ecg(signal)

print("Final shape sent:",
      len(processed_signal),
      "x",
      len(processed_signal[0]))

Final shape sent: 12 x 1000


In [31]:
# ==========================================
# STEP 5: CHECK API HEALTH
# ==========================================
print("\nChecking API health...")
health = requests.get(HEALTH_URL)
print("API Health:", health.json())

# ==========================================
# STEP 6: CLINICAL ANALYSIS
# ==========================================
print("\nSending ECG for clinical analysis...")

analyze_response = requests.post(
    ANALYZE_URL,
    json={"signal": processed_signal}
)

if analyze_response.status_code == 200:
    result = analyze_response.json()
    print("\nClinical Analysis Result:")
    print(result)
else:
    print("Analyze Error:", analyze_response.text)


Checking API health...
API Health: {'status': 'healthy', 'models_loaded': True, 'fairseq_signals_available': True, 'models': {'pretrained': True, 'finetuned': True}, 'timestamp': 1772171542.9978156}

Sending ECG for clinical analysis...

Clinical Analysis Result:
{'status': 'success', 'processing_time_ms': 210.7, 'clinical_analysis': {'rhythm': 'Tachycardia', 'heart_rate': None, 'qrs_duration': None, 'qt_interval': None, 'pr_interval': None, 'axis_deviation': 'Normal', 'abnormalities': ['Tachycardia'], 'confidence': 0.9535354375839233, 'confidence_level': 'High', 'review_required': True, 'probabilities': [0.000584907946176827, 0.20900027453899384, 0.0006691849557682872, 0.9535354375839233, 4.630748251089951e-23, 0.04039615020155907, 0.00044459200580604374, 1.3832652712153504e-07, 0.0005110920174047351, 0.013691852800548077, 0.0007550792652182281, 0.001508998917415738, 1.0857474990189075e-05, 0.04614996910095215, 0.18266046047210693, 0.07031852751970291, 0.043899424374103546], 'label_p

In [32]:
# ==========================================
# STEP 7: FEATURE EXTRACTION
# ==========================================
print("\nExtracting feature embeddings...")

feat_response = requests.post(
    FEATURE_URL,
    json={"signal": processed_signal}
)

if feat_response.status_code == 200:
    features = feat_response.json()
    print("Feature vector keys:", features.keys())
else:
    print("Feature Error:", feat_response.text)

print("\nPipeline completed successfully.")


Extracting feature embeddings...
Feature vector keys: dict_keys(['status', 'processing_time_ms', 'features', 'physiological_parameters', 'model_source'])

Pipeline completed successfully.


In [24]:
analyze_response = requests.post(
    "https://mystic-cbk-ecg-fm-api.hf.space/analyze",
    json={"signal": processed_signal}
)

print(analyze_response.json())

{'status': 'success', 'processing_time_ms': 185.65, 'clinical_analysis': {'rhythm': 'Tachycardia', 'heart_rate': None, 'qrs_duration': None, 'qt_interval': None, 'pr_interval': None, 'axis_deviation': 'Normal', 'abnormalities': ['Tachycardia'], 'confidence': 0.9535354375839233, 'confidence_level': 'High', 'review_required': True, 'probabilities': [0.000584907946176827, 0.20900027453899384, 0.0006691849557682872, 0.9535354375839233, 4.630748251089951e-23, 0.04039615020155907, 0.00044459200580604374, 1.3832652712153504e-07, 0.0005110920174047351, 0.013691852800548077, 0.0007550792652182281, 0.001508998917415738, 1.0857474990189075e-05, 0.04614996910095215, 0.18266046047210693, 0.07031852751970291, 0.043899424374103546], 'label_probabilities': {'Poor data quality': 0.000584907946176827, 'Sinus rhythm': 0.20900027453899384, 'Premature ventricular contraction': 0.0006691849557682872, 'Tachycardia': 0.9535354375839233, 'Ventricular tachycardia': 4.630748251089951e-23, 'Supraventricular tachy

In [23]:
# STEP 7: EXTRACT FEATURES
# ==========================================
print("\nExtracting ECG features...")

feat_response = requests.post(
    FEATURE_URL,
    json={"signal": processed_signal}
)

if feat_response.status_code == 200:
    features = feat_response.json()
    print("Feature vector length:", len(features.get("features", [])))
else:
    print("Feature Error:", feat_response.text)

print("\nPipeline completed successfully.")


Extracting ECG features...
Feature vector length: 3

Pipeline completed successfully.
